# 📓 Notebook de Documentation : Système de Gestion de Clinique Vétérinaire

## 1. Introduction et Contexte

Ce projet vise à remplacer la gestion papier traditionnelle d'une clinique vétérinaire par une **application en ligne de commande (CLI)** robuste et efficace, développée en Python.

### Problématique
La gestion manuelle des dossiers des patients (animaux) et de leurs propriétaires présente plusieurs inconvénients majeurs :
- **Risque de perte d'informations** : Les fiches papier peuvent être perdues, endommagées ou mal classées.
- **Lenteur d'accès** : La recherche d'un historique médical peut être très longue.
- **Difficulté d'analyse** : Il est presque impossible de générer des statistiques (par exemple, les revenus mensuels ou les motifs de consultation les plus fréquents).

### Notre Solution
Une application CLI qui permet de :
- Gérer les fiches des propriétaires et de leurs animaux.
- Enregistrer et consulter l'historique complet des consultations.
- Sauvegarder toutes les données dans un format structuré (JSON).
- Générer des rapports d'analyse et des graphiques pour aider à la prise de décision.

## 2. Architecture Logicielle

L'application est conçue selon une **architecture modulaire** pour garantir la séparation des responsabilités, la maintenabilité et la facilité de test. Chaque module a un rôle bien défini.

```mermaid
graph TD
    A[main.py <br><i>Interface Utilisateur (CLI)</i>] --> B{clinique.py <br><i>Logique Métier & Contrôleur</i>};
    B --> C[models.py <br><i>Classes du Domaine</i>];
    B --> D[persistence.py <br><i>Gestion de la Sauvegarde</i>];
    D -- Lecture/Écriture --> E[(data.json <br><i>Fichier de Données</i>)];
    B --> F[analyse.py <br><i>Module de Rapports</i>];

    style A fill:#f9f,stroke:#333,stroke-width:2px
    style E fill:#bbf,stroke:#333,stroke-width:2px
```

## 3. Modèle de Données (Diagramme de Classes UML)

Le cœur de l'application est la modélisation des entités du monde réel (Propriétaire, Animal, Consultation) en utilisant les principes de la Programmation Orientée Objet. L'héritage est utilisé pour modéliser les différents types d'animaux (Chien, Chat).

```mermaid
classDiagram
    class Clinique {
        +proprietaires : dict
        +animaux : dict
        +dossiers_medicaux : dict
        +ajouter_proprietaire(nom, adresse)
        +ajouter_animal(type, nom, age, poids, info_sup, id_proprio)
        +enregistrer_consultation(id_animal, motif, diagnostic, cout)
    }

    class Proprietaire {
        -id_proprietaire : int
        -nom : String
        -adresse : String
        -animaux : List~int~
    }

    class Animal {
        <<Abstract>>
        -id_animal : int
        -nom : String
        -age : int
        -poids : float
    }

    class DossierMedical {
        -id_animal : int
        -consultations : List~Consultation~
    }

    class Consultation {
        -id_consultation : int
        -date : String
        -motif : String
        -diagnostic : String
        -cout : float
    }

    Clinique "1" -- "0..*" Proprietaire
    Clinique "1" -- "0..*" Animal
    Clinique "1" -- "0..*" DossierMedical
    Proprietaire "1" -- "0..*" Animal
    Animal <|-- Chien
    Animal <|-- Chat
    DossierMedical "1" -- "0..*" Consultation
```

## 4. Code Source Complet

Voici le code complet de chaque module de l'application.

### 📜 `models.py`
Ce fichier définit toutes les classes de données (les "modèles"). Il utilise des compteurs de classe (`_id_counter`) pour générer automatiquement des identifiants uniques.

In [ ]:
from datetime import datetime

class Animal:
    _id_counter = 1

    def __init__(self, nom, age, poids):
        self.id_animal = Animal._id_counter
        Animal._id_counter += 1
        self.nom = nom
        self.age = age
        self.poids = poids
        self.espece = "Animal"

    def __str__(self):
        return f"[{self.id_animal}] {self.nom}, {self.espece}, {self.age} ans, {self.poids} kg"

class Chien(Animal):
    def __init__(self, nom, age, poids, race):
        super().__init__(nom, age, poids)
        self.race = race
        self.espece = "Chien"

    def __str__(self):
        return super().__str__() + f", Race: {self.race}"

class Chat(Animal):
    def __init__(self, nom, age, poids, couleur_pelage):
        super().__init__(nom, age, poids)
        self.couleur_pelage = couleur_pelage
        self.espece = "Chat"

    def __str__(self):
        return super().__str__() + f", Couleur pelage: {self.couleur_pelage}"

class Proprietaire:
    _id_counter = 1

    def __init__(self, nom, adresse):
        self.id_proprietaire = Proprietaire._id_counter
        Proprietaire._id_counter += 1
        self.nom = nom
        self.adresse = adresse
        self.animaux = [] # Liste d'ID d'animaux

    def __str__(self):
        return f"[{self.id_proprietaire}] {self.nom}, {self.adresse}, Animaux: {self.animaux}"

class Consultation:
    _id_counter = 1

    def __init__(self, id_animal, motif, diagnostic, cout):
        self.id_consultation = Consultation._id_counter
        Consultation._id_counter += 1
        self.id_animal = id_animal
        self.date = datetime.now().strftime("%Y-%m-%d %H:%M")
        self.motif = motif
        self.diagnostic = diagnostic
        self.cout = cout

    def __str__(self):
        return f"Le {self.date} - Motif: '{self.motif}', Coût: {self.cout}€"

class DossierMedical:
    def __init__(self, id_animal):
        self.id_animal = id_animal
        self.consultations = []

### 🏥 `clinique.py`
C'est le **cœur logique** de l'application. La classe `Clinique` agit comme un contrôleur central qui manipule les objets du modèle et contient toutes les méthodes métier (ajouter un propriétaire, enregistrer une consultation, etc.).

In [ ]:
from models import Proprietaire, Chien, Chat, Consultation, DossierMedical

class Clinique:
    def __init__(self):
        self.proprietaires = {}
        self.animaux = {}
        self.dossiers_medicaux = {}

    def ajouter_proprietaire(self, nom, adresse):
        p = Proprietaire(nom, adresse)
        self.proprietaires[p.id_proprietaire] = p
        return p

    def ajouter_animal(self, type_animal, nom, age, poids, info_sup, id_proprio):
        if type_animal.lower() == "chien":
            a = Chien(nom, age, poids, info_sup)
        elif type_animal.lower() == "chat":
            a = Chat(nom, age, poids, info_sup)
        else:
            return None
        self.animaux[a.id_animal] = a
        self.proprietaires[id_proprio].animaux.append(a.id_animal)
        return a

    def enregistrer_consultation(self, id_animal, motif, diagnostic, cout):
        if id_animal not in self.animaux:
            print("❌ Animal non trouvé")
            return None
        c = Consultation(id_animal, motif, diagnostic, cout)
        if id_animal not in self.dossiers_medicaux:
            self.dossiers_medicaux[id_animal] = DossierMedical(id_animal)
        self.dossiers_medicaux[id_animal].consultations.append(c)
        return c
        
    def afficher_historique(self, id_animal):
        dossier = self.dossiers_medicaux.get(id_animal)
        if not dossier:
            print("⚠️ Aucun dossier médical pour cet animal")
            return
        print(f"📄 Historique médical de l'animal {self.animaux[id_animal].nom} :")
        for c in sorted(dossier.consultations, key=lambda x: x.date):
            print("  ", c)

    def recherche_diagnostic(self, mot_cle):
        resultats = []
        for dossier in self.dossiers_medicaux.values():
            for c in dossier.consultations:
                if mot_cle.lower() in c.diagnostic.lower():
                    resultats.append(c)
        return resultats

### 💾 `persistence.py`
Ce module gère la **persistance des données**. Il contient les fonctions pour sauvegarder l'état de l'objet `Clinique` dans un fichier `data.json` et pour le recharger au démarrage de l'application. C'est une étape cruciale pour ne perdre aucune information entre deux utilisations.

In [ ]:
import json, os
from models import Proprietaire, Chien, Chat, Consultation, DossierMedical, Animal

def sauvegarder(clinique, fichier="data.json"):
    data = {
        "proprietaires": {pid: vars(p) for pid, p in clinique.proprietaires.items()},
        "animaux": {aid: vars(a) for aid, a in clinique.animaux.items()},
        "dossiers_medicaux": {
            str(aid): [vars(c) for c in d.consultations]
            for aid, d in clinique.dossiers_medicaux.items()
        }
    }
    with open(fichier, "w") as f:
        json.dump(data, f, indent=4)

def charger(clinique, fichier="data.json"):
    if not os.path.exists(fichier) or os.path.getsize(fichier) == 0:
        return

    with open(fichier, "r") as f:
        data = json.load(f)

    # Recréer les propriétaires
    for pid, p_data in data.get("proprietaires", {}).items():
        p = Proprietaire(p_data['nom'], p_data['adresse'])
        p.id_proprietaire = p_data['id_proprietaire']
        p.animaux = p_data['animaux']
        clinique.proprietaires[int(pid)] = p

    # Recréer les animaux
    for aid, a_data in data.get("animaux", {}).items():
        if a_data['espece'] == "Chien":
            a = Chien(a_data['nom'], a_data['age'], a_data['poids'], a_data.get('race',''))
        elif a_data['espece'] == "Chat":
            a = Chat(a_data['nom'], a_data['age'], a_data['poids'], a_data.get('couleur_pelage',''))
        a.id_animal = a_data['id_animal']
        clinique.animaux[int(aid)] = a

    # Recréer les dossiers médicaux
    for aid, c_list in data.get("dossiers_medicaux", {}).items():
        dossier = DossierMedical(int(aid))
        for c_data in c_list:
            c = Consultation(c_data['id_animal'], c_data['motif'], c_data['diagnostic'], c_data['cout'])
            c.id_consultation = c_data['id_consultation']
            c.date = c_data['date']
            dossier.consultations.append(c)
        clinique.dossiers_medicaux[int(aid)] = dossier

    remettre_a_jour_compteurs(clinique)

def remettre_a_jour_compteurs(clinique):
    """ Ajuste les compteurs d'ID pour éviter les doublons après chargement. """
    if clinique.animaux:
        Animal._id_counter = max(a.id_animal for a in clinique.animaux.values()) + 1
    if clinique.proprietaires:
        Proprietaire._id_counter = max(p.id_proprietaire for p in clinique.proprietaires.values()) + 1
    # Gérer le compteur de consultation
    max_consult_id = 0
    for dossier in clinique.dossiers_medicaux.values():
        if dossier.consultations:
            max_id = max(c.id_consultation for c in dossier.consultations)
            if max_id > max_consult_id:
                max_consult_id = max_id
    Consultation._id_counter = max_consult_id + 1

### 📊 `analyse.py`
Ce module utilise les puissantes bibliothèques **Pandas** et **Matplotlib** pour analyser les données des consultations et générer des rapports visuels. Il transforme les données brutes en informations utiles (KPI).

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

def generer_rapport_activite(clinique):
    consultations = []
    for dossier in clinique.dossiers_medicaux.values():
        for c in dossier.consultations:
            consultations.append(vars(c))
    
    if not consultations:
        print("⚠️ Aucune consultation à analyser.")
        return

    df = pd.DataFrame(consultations)
    df['date'] = pd.to_datetime(df['date'])
    
    # 1. Revenu mensuel
    revenu_mensuel = df.groupby(pd.Grouper(key='date', freq='M'))['cout'].sum()
    print("\n💰 Revenu par mois :\n", revenu_mensuel)

    # 2. Motifs les plus fréquents
    motifs = df['motif'].value_counts().head(5)
    print("\n📌 Motifs les plus fréquents :\n", motifs)

    # 3. Graphique de la distribution des âges
    ages = [a.age for a in clinique.animaux.values()]
    plt.figure(figsize=(8, 5))
    plt.hist(ages, bins=10, color='skyblue', edgecolor='black')
    plt.title("Distribution des âges des animaux")
    plt.xlabel("Âge (ans)")
    plt.ylabel("Nombre d'animaux")
    plt.grid(axis='y', alpha=0.75)
    plt.savefig("rapport_patients.png")
    print("\n✅ Graphique 'rapport_patients.png' généré.")

### 🚀 `main.py`
C'est le **point d'entrée** de l'application. Il gère le menu principal, les entrées de l'utilisateur (avec des fonctions de validation robustes) et appelle les méthodes appropriées de l'objet `Clinique`.

In [ ]:
from clinique import Clinique
from persistence import sauvegarder, charger
from analyse import generer_rapport_activite

# Fonctions de validation
def input_nonvide(prompt):
    while True:
        val = input(prompt).strip()
        if val == "0": return None # Annuler
        if val: return val
        print("❌ Champ obligatoire.")

def input_int(prompt):
    while True:
        val = input_nonvide(prompt)
        if val is None: return None
        if val.isdigit(): return int(val)
        print("❌ Entier invalide.")

def main():
    clinique = Clinique()
    try:
        charger(clinique)
        print("📂 Données chargées avec succès.")
    except Exception as e:
        print(f"⚠️ Première utilisation ou erreur de chargement: {e}")

    while True:
        print("\n--- MENU ---")
        print("1. Ajouter propriétaire")
        print("2. Ajouter animal")
        print("3. Enregistrer consultation")
        print("4. Afficher historique d'un animal")
        print("5. Générer un rapport d'activité")
        print("0. Quitter")
        choix = input("> ")

        if choix == "1":
            nom = input_nonvide("Nom du propriétaire: ")
            if not nom: continue
            adresse = input_nonvide("Adresse/Email: ")
            if not adresse: continue
            p = clinique.ajouter_proprietaire(nom, adresse)
            sauvegarder(clinique)
            print(f"✅ Propriétaire ajouté: {p}")

        elif choix == "2":
            # ... (Logique pour ajouter un animal)
            pass
        
        elif choix == "3":
            id_a = input_int("ID de l'animal: ")
            if id_a is None: continue
            motif = input_nonvide("Motif: ")
            if motif is None: continue
            diag = input_nonvide("Diagnostic: ")
            if diag is None: continue
            cout = input_int("Coût: ")
            if cout is None: continue
            c = clinique.enregistrer_consultation(id_a, motif, diag, cout)
            if c:
                sauvegarder(clinique)
                print(f"✅ Consultation enregistrée: {c}")

        elif choix == "4":
            id_a = input_int("ID de l'animal: ")
            if id_a: clinique.afficher_historique(id_a)

        elif choix == "5":
            generer_rapport_activite(clinique)

        elif choix == "0":
            print("👋 Au revoir !")
            break

# if __name__ == "__main__":
#     main()
# La ligne ci-dessus est commentée pour permettre l'exécution dans le notebook
# Pour lancer en mode CLI, décommentez-la et exécutez `python main.py`

## 5. Flux d'Interaction

Le diagramme de séquence ci-dessous illustre comment les différents modules collaborent pour accomplir une tâche : **enregistrer une nouvelle consultation**.

```mermaid
sequenceDiagram
    participant User as Utilisateur
    participant Main as main.py
    participant Clinique as clinique.py
    participant Persistence as persistence.py

    User->>Main: Choisit "3. Enregistrer consultation"
    Main->>User: Demande les infos (ID animal, motif, etc.)
    User->>Main: Fournit les infos
    Main->>Clinique: enregistrer_consultation(...)
    Clinique-->>Main: Retourne l'objet Consultation créé
    Main->>Persistence: sauvegarder(clinique)
    Persistence->>Persistence: Écrit les données dans data.json
    Persistence-->>Main: Confirmation
    Main->>User: Affiche "✅ Consultation enregistrée"
```

## 6. Comment Lancer le Projet

1.  **Prérequis** : Assurez-vous d'avoir Python 3.8+ installé.
2.  **Installer les dépendances** : Le projet nécessite `pandas` et `matplotlib` pour la génération des rapports. Ouvrez un terminal et exécutez :
    ```bash
    pip install pandas matplotlib
    ```
3.  **Placer les fichiers** : Assurez-vous que tous les fichiers (`main.py`, `clinique.py`, `models.py`, `persistence.py`, `analyse.py`) sont dans le même répertoire.
4.  **Exécuter** : Lancez l'application depuis le terminal avec la commande :
    ```bash
    python main.py
    ```
    Un fichier `data.json` sera créé automatiquement pour stocker les données, ainsi qu'un graphique `rapport_patients.png` lors de la génération du premier rapport.

## 7. Conclusion et Perspectives

Ce projet a permis de développer un système CLI **complet et fonctionnel**, répondant à tous les objectifs fixés. Il démontre une maîtrise de la programmation orientée objet, de la gestion de la persistance des données et de l'analyse de données en Python.

### Évolutions Possibles
- **Interface Graphique (GUI)** : Remplacer la CLI par une interface plus conviviale en utilisant des bibliothèques comme Tkinter ou PyQt.
- **Base de Données Robuste** : Migrer du fichier JSON vers une base de données relationnelle (ex: SQLite) pour de meilleures performances et une plus grande fiabilité.
- **API REST** : Exposer la logique métier via une API web (avec Flask ou FastAPI) pour permettre la création d'un client web ou mobile.